In [1]:
# 自动计算cell的计算时间
%load_ext autotime

%matplotlib inline
%config InlineBackend.figure_format='svg' #矢量图设置，让绘图更清晰

%reload_ext tensorboard

time: 305 ms (started: 2021-09-01 18:18:08 +08:00)


In [2]:
%%bash

# 增加更新
git add *.ipynb */*.ipynb

git remote -v

git commit -m '更新 #2 Sept 01, 2021'

#git push origin master
git push

origin	git@github.com:ustchope/pytorch_lightning_study.git (fetch)
origin	git@github.com:ustchope/pytorch_lightning_study.git (push)
[main 2db8b76] 更新 #2 Sept 01, 2021
 6 files changed, 2938 insertions(+), 3 deletions(-)
 create mode 100644 "\346\225\231\347\250\213/.ipynb_checkpoints/PL CIFAR10 ~94% \345\237\272\347\272\277\346\225\231\347\250\213-checkpoint.ipynb"
 create mode 100644 "\346\225\231\347\250\213/.ipynb_checkpoints/PYTORCH LIGHTNING DATAMODULES-checkpoint.ipynb"
 create mode 100644 "\346\225\231\347\250\213/PL CIFAR10 ~94% \345\237\272\347\272\277\346\225\231\347\250\213.ipynb"
 create mode 100644 "\346\225\231\347\250\213/PYTORCH LIGHTNING DATAMODULES.ipynb"


To github.com:ustchope/pytorch_lightning_study.git
   ca7ee2c..2db8b76  main -> main


time: 3.79 s (started: 2021-09-01 18:18:16 +08:00)


In [11]:
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from pl_bolts.datamodules import CIFAR10DataModule
from pl_bolts.transforms.dataset_normalizations import cifar10_normalization
from pytorch_lightning import LightningModule, seed_everything, Trainer, LightningDataModule
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
from torch.optim.lr_scheduler import OneCycleLR
from torch.optim.swa_utils import AveragedModel, update_bn
from torchmetrics.functional import accuracy
import torchmetrics

seed_everything(7)

PATH_DATASETS = os.environ.get('PATH_DATASETS', '.')
AVAIL_GPUS = min(1, torch.cuda.device_count())
BATCH_SIZE = 256 if AVAIL_GPUS else 64
NUM_WORKERS = int(os.cpu_count() / 2)

time: 1.83 ms (started: 2021-09-01 18:23:32 +08:00)


# CIFAR10 数据模块

从`bots`导入现有数据模块并修改训练和测试转换。

In [10]:
train_transforms = torchvision.transforms.Compose([
    torchvision.transforms.RandomCrop(32, padding=4),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor(),
    cifar10_normalization(),
])

test_transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    cifar10_normalization(),
])

cifar10_dm = CIFAR10DataModule(
    data_dir=PATH_DATASETS,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    train_transforms=train_transforms,
    test_transforms=test_transforms,
    val_transforms=test_transforms,
)

time: 2.13 ms (started: 2021-09-01 18:23:24 +08:00)


# 网络

从 TorchVision 修改预先存在的 Resnet 架构。 预先存在的架构基于 ImageNet 图像 (224x224) 作为输入。 所以我们需要针对 CIFAR10 图像（32x32）修改它。

In [24]:
def create_model():
    model = torchvision.models.resnet18(pretrained=False, num_classes=10)
    model.conv1 = nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    model.maxpool = nn.Identity()
    return model

time: 1.08 ms (started: 2021-09-01 18:31:22 +08:00)


## PL模块

查看 `configure_optimizers <https://pytorch-lightning.readthedocs.io/en/stable/common/lightning_module.html#configure-optimizers>`__ 方法以使用自定义学习率调度程序。 带有 SGD 的 OneCycleLR 将使您在 20-30 个时期内达到大约 92-93% 的准确度，在 40-50 个时期内达到 93-94% 的准确度。 随意尝试来自 https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate 的不同 LR 计划

In [22]:
class LitResnet(LightningModule):

    def __init__(self, lr=0.05):
        super().__init__()

        self.save_hyperparameters()
        self.model = create_model()

    def forward(self, x):
        out = self.model(x)
        return F.log_softmax(out, dim=1)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        self.log('train_loss', loss)
        return loss

    def evaluate(self, batch, stage=None):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, y)

        if stage:
            self.log(f'{stage}_loss', loss, prog_bar=True)
            self.log(f'{stage}_acc', acc, prog_bar=True)

    def validation_step(self, batch, batch_idx):
        self.evaluate(batch, 'val')

    def test_step(self, batch, batch_idx):
        self.evaluate(batch, 'test')

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(
            self.parameters(),
            lr=self.hparams.lr,
            momentum=0.9,
            weight_decay=5e-4,
        )
        steps_per_epoch = 45000 // BATCH_SIZE
        scheduler_dict = {
            'scheduler': OneCycleLR(
                optimizer,
                0.1,
                epochs=self.trainer.max_epochs,
                steps_per_epoch=steps_per_epoch,
            ),
            'interval': 'step',
        }
        return {'optimizer': optimizer, 'lr_scheduler': scheduler_dict}

time: 2.18 ms (started: 2021-09-01 18:30:31 +08:00)


In [28]:
model = LitResnet(lr=0.05)
model.datamodule = cifar10_dm

trainer = Trainer(
    progress_bar_refresh_rate=10,
    max_epochs=30, accelerator='dp',
    gpus=4,
    logger=TensorBoardLogger('lightning_logs/', name='resnet'),
    callbacks=[LearningRateMonitor(logging_interval='step')],
)

trainer.fit(model, cifar10_dm)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/data/miniconda3/envs/torch19/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:423: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7,8,9]

  | Name  | Type   | Params
---------------------------------
0 | model | ResNet | 11.2 M
---------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.696    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

time: 15min 18s (started: 2021-09-01 18:35:41 +08:00)


/data/miniconda3/envs/torch19/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:423: LightningDeprecationWarning: DataModule.teardown has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.teardown.
  rank_zero_deprecation(


In [29]:
trainer.test(model, datamodule=cifar10_dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7,8,9]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.9193515777587891, 'test_loss': 0.2936635911464691}
--------------------------------------------------------------------------------


[{'test_loss': 0.2936635911464691, 'test_acc': 0.9193515777587891}]

time: 6.71 s (started: 2021-09-01 18:50:59 +08:00)


# 奖励：使用随机权重平均来提高性能

使用 torch.optim 中的 SWA 获得快速的性能提升。 还展示了 Lightning 的一些很酷的功能： - 使用 training_epoch_end 在每个 epoch 结束后运行代码 - 直接使用预训练模型与此包装器一起用于 SWA

In [30]:
class SWAResnet(LitResnet):

    def __init__(self, trained_model, lr=0.01):
        super().__init__()

        self.save_hyperparameters('lr')
        self.model = trained_model
        self.swa_model = AveragedModel(self.model)

    def forward(self, x):
        out = self.swa_model(x)
        return F.log_softmax(out, dim=1)

    def training_epoch_end(self, training_step_outputs):
        self.swa_model.update_parameters(self.model)

    def validation_step(self, batch, batch_idx, stage=None):
        x, y = batch
        logits = F.log_softmax(self.model(x), dim=1)
        loss = F.nll_loss(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, y)

        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(
            self.model.parameters(), lr=self.hparams.lr, momentum=0.9, weight_decay=5e-4
        )
        return optimizer

    def on_train_end(self):
        update_bn(self.datamodule.train_dataloader(), self.swa_model, device=self.device)

time: 1.24 ms (started: 2021-09-01 18:51:06 +08:00)


In [31]:
swa_model = SWAResnet(model.model, lr=0.01)
swa_model.datamodule = cifar10_dm

swa_trainer = Trainer(
    progress_bar_refresh_rate=20,
    max_epochs=20,
    gpus=AVAIL_GPUS,
    logger=TensorBoardLogger('lightning_logs/', name='swa_resnet'),
)

swa_trainer.fit(swa_model, cifar10_dm)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/data/miniconda3/envs/torch19/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:423: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7,8,9]

  | Name      | Type          | Params
--------------------------------------------
0 | model     | ResNet        | 11.2 M
1 | swa_model | AveragedModel | 11.2 M
--------------------------------------------
22.3 M    Trainable params
0         Non-trainable params
22.3 M    Total params
89.392    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

<ipython-input-30-0f2e250a02cf>:34: LightningDeprecationWarning: The `LightningModule.datamodule` property is deprecated in v1.3 and will be removed in v1.5. Access the datamodule through using `self.trainer.datamodule` instead.
  update_bn(self.datamodule.train_dataloader(), self.swa_model, device=self.device)


time: 12min 9s (started: 2021-09-01 18:51:06 +08:00)


/data/miniconda3/envs/torch19/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:423: LightningDeprecationWarning: DataModule.teardown has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.teardown.
  rank_zero_deprecation(


In [32]:
swa_trainer.test(swa_model, datamodule=cifar10_dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7,8,9]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.9190999865531921, 'test_loss': 0.261920690536499}
--------------------------------------------------------------------------------


[{'test_loss': 0.261920690536499, 'test_acc': 0.9190999865531921}]

time: 6.08 s (started: 2021-09-01 19:03:16 +08:00)


In [ ]:
# Start tensorboard.
%reload_ext tensorboard
%tensorboard --logdir lightning_logs/